## Importing necessary packages 

In [1]:
import pandas as pd
import plotly.express as px
from pandas_datareader import wb      
# import pandas_datareader.data as web  
# from datetime import datetime, timedelta

from jupyter_dash import JupyterDash
from dash import dcc, html

import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

from dash.dependencies import Input, Output

# import requests 

dbc_css = 'https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates@V1.0.2/dbc.min.css'
load_figure_template('bootstrap')

# Task 1 

In [ ]:
# Retrieve data 
data = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')

# look at data 
data.head()

Working with tidying the data follows, I observe there are some columns not needed for the analysis, as the task says. 

In [ ]:
# Extract relevant columns 
data = data[['iso_code','location', 'continent', 'date', 'new_cases', 
      'total_cases', 'total_cases_per_million', 'new_deaths', 
      'total_deaths', 'total_deaths_per_million', 'new_vaccinations', 
      'people_fully_vaccinated', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred']]


In [ ]:
# Checking classes 
data.info()

# Converting to datetime 
data['date'] = pd.to_datetime(data['date'])

In [ ]:
# data.info()

## Feature 1 

Considering that this is data about the world as a whole that we want to retrieve, we can use the info located where location equals world in the dataset. 

In [ ]:
# We have a country called world we can use for the first feature 
world_df = data[data['location'] == 'World']

# Sorting in descending order - leaving the values we want on the top row of the dataset. 
world_df = world_df.sort_values(by = 'date', ascending = False)

# Limiting to the columns of interest. 
world_df = world_df[['location' , 'date', 'total_deaths','total_cases', 'people_fully_vaccinated']]



world_df.head()

In [ ]:
# Extracting the values of interest and storing them in variables that we supply to the first card 
# in our application 
TotalDeaths = round(world_df.iloc[0]['total_deaths'])
TotalCases = round(world_df.iloc[0]['total_cases'])
PeopleVaccinatedFully = round(world_df.iloc[1]['people_fully_vaccinated'])
PeopleVaccinatedFully

DateOfToday = world_df.iloc[0]['date']
DateOfToday

## Feature 2 

I interpret this task as that if the user selects Deaths as a variable, only the total, total per 1 million and newly reported 
will appear for the deaths variable, and not for the cases variable. Same goes for cases. This gives the user the availability to choose between three metrics after initial choice of variable. 

In [ ]:
# Removing world location 
df_feature2 = data[data['location'] != 'World']

#changing the location column to country - not really necessary but done for clarity.
df_feature2.rename(columns = {'location': 'country'}, inplace = True)



In [ ]:
df_feature2.head()

In [ ]:
# Checking for weird continents, seems to make sense. 
df_feature2['continent'].value_counts()

Note! I further work with the dataset in a way in which I perceived that summarising the dataset did not make sense because 
of aggregated columns. One can not summarise the total deaths i.e., because this one is not changing from day to day, leaving the initial 
summarised value way too large. 

Therefore, I choose three dates to work with. 
This is: 
- 31/12/2020 -> for visualising the first year
- 31/12/2021 -> for visualising the first and second year
- 22/05/2022 -> for visualising the first, second and third year

In [ ]:
# These variables are later used in the application for subsetting 
year1 = df_feature2[df_feature2['date'] == '2020-12-31'].copy()
year2 = df_feature2[df_feature2['date'] == '2021-12-31'].copy()
total_df = df_feature2[df_feature2['date'] == '2022-05-22'].copy()

# total_df = df_feature2[df_feature2['date'] == '2022-05-22'].copy()



In [ ]:
# These are the final values in the dataset 
total_df.head()

In [ ]:
# Variables for storing the dates
date_year1 = '2020-12-31'
date_year2 = '2021-12-31'
date_year3 = '2022-05-22'

See comments further down in the application regarding how I implemented feature 2. 

## Feature number 3 

The global number of occurences is interpreted that it is the world numbers the task is looking for. I.e, in similarity with 
feature 1, I can use the dataframe where location equals world. 

In [ ]:
# barplot for each week 
weekly_df = data 
# weekly_df.info()

In [ ]:

# This function is defined to change our dataframe data into what is needed for the application. 
# By filtering out where location equals world, reseting index, resampling by weeking and summarising, 
# the new data seems ready for the application 
def changing_to_weekly(df = weekly_df): 
    
    df = df[df['location'] == 'World'].reset_index().set_index('date').resample('w').sum().reset_index()
    df = df[['date', 'new_deaths','new_cases','new_vaccinations']]
    
    # renaming for clarity
    df.rename(columns = {'date': 'week'}, inplace = True)
    
    return df

In [ ]:
weekly_df = changing_to_weekly(weekly_df)

# Contains what we need for the barplot 
weekly_df.head()

In [ ]:
weekly_df.head()

In [ ]:
# Renaming so labels get nicer 
weekly_df.rename(columns = {'new_deaths':'New Deaths',
                            'new_cases':'New Cases',
                            'new_vaccinations':'New Vaccinations'}, inplace = True)


# Application development

## Dropdown menus and radioitems used in the application 

In [ ]:
# options for the dropdown below
options_variable = [{'label': 'Deaths', 'value':'total_deaths'}, 
                    {'label':'Cases', 'value': 'total_cases'},
                    {'label': 'Vaccinations', 'value': 'total_vaccinations_per_hundred'}
]

# Variable of choice 
var_choice = dcc.Dropdown(
    id = 'my_variable',
    options = options_variable,
    value = None # No default value 
)
#-----------------------------------------

# the dependent metric (is filled with the choice the user makes in the dropdown above)
dependent_metric = dcc.Dropdown(
    id = 'my_metric',
    options = [], # empty list 
    value = None # default = None, because this is dependent on the variable choice
)


#-----------------------------------------

# variable of choice for bar plot, using radioitems here. Could might as well have used dropdown here too. 
bar_button = dcc.RadioItems(
    id = 'my_bar_choice',
    
    # options equals the variable names for the columns in the weekly df, 
    # except for the one named week, naturally
    options = [{'label': i, 'value': i} for i in weekly_df.columns[1:]], 
    value = 'New Cases'
)


## Cards: 
I split my application into cards and tabs, because I feel that it makes it a lot more nicer.

In [ ]:
# Welcome card as my "front page"
welcome_card = dbc.Card(
    children = [  

        # 1st row with card header
        dbc.Row([

            # column with shool's logo
            dbc.Col(dbc.CardImg(src = 'assets/nhh.png', style = {'height' : 40}), width = 'auto'), # picture 

            # column with div information
            dbc.Col(html.H2('BAN438 Exam | Candidate: 6',style = {'textAlign' : 'center'}), width = 9)
            
        ]),
        
    ], 
    body = True 
)

In [ ]:
## Feature 1 
cardFeature1 = dbc.Card(
    children = [  

        # 1st row with 
        dbc.Row(dbc.Col(html.H4('Feature 1: ', className = 'card-title'), width = "auto")), # title 
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.P(f'Note! These values are static and therefore only valid for the last date in the dataframe, which is ({DateOfToday}).', className = 'card-text'))),
                # description above 
        
            
        html.Br(), 
        dbc.Row(children = [
            dbc.Col(html.H4(f'Total deaths: {TotalDeaths:,}', style = {'color' : 'darkred'}), width = 4), # number contains thousands separator
            html.Br(),
            dbc.Col(html.H4(f'Total cases: {TotalCases:,}', style = {'color' : 'red'}), width = 4),
            html.Br(),
            dbc.Col(html.H4(f'People fully vaccinated: {PeopleVaccinatedFully:,}', 
                            style = {'color' : 'green'}), width = 4),
                     ]
                
               )
        
    ],
    body = True
                

    
)

In [ ]:
# Choropleth card - containing two dropdowns and the graph. 
card_choropleth = dbc.Card(children = [
    dbc.Row(dbc.Col(html.H4('Feature 2: ', className = 'card-title'))),
    dbc.Row(dbc.Col(html.P('Note! The variable of interest must be chosen before the metric and graph appears: ', className = 'card-text'))),
    
    html.Br(), 
    
    # Variable choice 
    dbc.Row(children = [
        dbc.Col(html.H4('Choose variable of interest: '), width = 4),
        dbc.Col(var_choice, width = 4)]),
        
    html.Br(), 
    
    # Metric choice 
    dbc.Row(children = [
        dbc.Col(html.H4('Choose metric of interest: '), width = 4),
        dbc.Col(dependent_metric)], style = {'display': 'none'}, # display is set to none so it will not be
                                                                # possible to choose metric before variable  
           id = 'show_metric'),
#    
    
    # this dcc input below is something I added extra. It lets the user choose what year to look at, 
    # whether it is 2020, 2021 or 2022 (which is the sum of all years). 
    # Note that the values displayed are the aggregate totals - so it is not possible to "compare years" in that sense. 
    # This is due to the fact that I interpreted it so that the task was not interested in the new cases each year. 
    
    # Input year (optional - inspiration retrieved from Plotly Dash website)
    html.Br(),
    dbc.Row(children = [
            dbc.Col(html.H4('Optional choice of year (2020-2022): '), width = "6"),
        dbc.Col(html.P('Notice that these are aggregated years. In example, a chosen year of 2021 equals the numbers from both 2020 and 2021. ', className = 'card-text')),
        
            dbc.Col(dcc.Input(id = 'year_input'.format('number'), 
                              type = "{}".format('number'),
                              
                              placeholder = 2022, # default 
                              size = "20", 
                                debounce=True, min = 2020, max = 2022, step = 1), width = 6)], 
           id = 'show_year'), # id because it will be displayed after user makes a choice regarding variable. 
    
    html.Br(),
    dbc.Row(dcc.Graph(id = 'my_choropleth'), style = {'display': 'none'}, id = 'show_graph'), 
    
    
    
], body = True
                     )


# card for the barplot 
card_barplot = dbc.Card(children = [
    dbc.Row(dbc.Col(html.H4('Feature 3: ', className = 'card-title'))),
    html.Br(), 
    dbc.Row(dbc.Col(bar_button, width = "8")), 
    html.Br(), 
    dbc.Row(dbc.Col(dcc.Graph(id = 'my_barplot'))),
    html.Br()], body = True
                     )

In [ ]:
## Insert the two cards above into tabs. This is a decision I made, could also have the cards stacked (i.e all cards on same 
# page)
tab1 = dbc.Tab(children = [card_choropleth], label = "Choropleth-tab")

tab2 = dbc.Tab(children = [card_barplot], label = "Bar-plot tab")
    

## The app layout

In [ ]:

## The app ## 
load_figure_template('lux')
app = JupyterDash(external_stylesheets = [dbc.themes.LUX, dbc_css])

app.layout = dbc.Container(
    children = [
        
        # insert card in column
        dbc.Row(dbc.Col(welcome_card, width = 12)),
        html.Br(), 
        cardFeature1,
        
        html.Br(), 
        dbc.Tabs(children = [
            tab1, 
            tab2
        ]), 
                 
        html.Br(), 
        html.Br() 
        
        

        
    ],
    className = 'dbc'
)

# ----------------------------------------------
# Callback for showing metric once variable is chosen by the user. 
# Inspiration for this is retrieved from the solution proposal for workshop 7
@app.callback(
   Output('show_metric', 'style'),
   Input('my_variable', 'value')
)


# Method retrieved from solution proposal to workshop 7
def show_metric(year): 
    if year == None: 
        return {'display': 'none'} 
    else: 
        return {'display': 'inline'}


# ----------------------------------------------
# Callback for showing the graph once both options is chosen by the user. 
    
@app.callback(
   Output('show_graph', 'style'),
   Input('my_metric', 'value'),
    Input('my_variable', 'value')
)

def show_my_graph(value1, value2): 
    if ((value1 == None)|(value2 == None)): 
        return {'display': 'none'} 
    else: 
        return {'display': 'inline'}


# ----------------------------------------------
# Callback for showing the optional year input once metric is chosen by the user. 

@app.callback(
   Output('show_year', 'style'),
   Input('my_metric', 'value'), 
    Input('my_variable', 'value'),
)

def show_year(value1, value2): 
    if ((value1 == None)|(value2 == None)): 
        return {'display': 'none'} 
    else: 
        return {'display': 'inline'}

    
# ----------------------------------------------
# Callback for linking the options for metric after variable is chosen by the user. 
@app.callback(
    Output('my_metric', 'options'),
    Input('my_variable', 'value'),
)

def choose_metric(value): 
    if (value ==  'total_deaths'):
        returning_value = [{'label': 'Total Deaths', 'value': 'total_deaths'},
                           {'label': 'Total Deaths per million', 'value': 'total_deaths_per_million'},
                           {'label': 'Newly reported deaths (last 24 hrs)', 'value': 'new_deaths'}]
                           
    
    elif (value == 'total_cases'):
         returning_value = [
                           {'label': 'Total Cases', 'value': 'total_cases'},
                           {'label': 'Total Cases per million', 'value': 'total_cases_per_million'},
                           {'label': 'Newly reported cases (last 24 hrs)', 'value': 'new_cases'}]
        
    
    else: 
        returning_value = [{'label': 'Total doses adms pr 100 pop', 'value': 'total_vaccinations_per_hundred'},
                           {'label': 'Total people vaccinated with at least one dose pr 100 pop', 
                            'value': 'people_vaccinated_per_hundred'},
                           {'label': 'Persons fully vaccinated', 'value': 'people_fully_vaccinated'}]
        
    return returning_value




# ----------------------------------------------
# Callback for launching the choropleth graph the graph once metric is chosen by the user. 
@app.callback(
    Output('my_choropleth', 'figure'),
    Input('my_metric', 'value'),
    Input('year_input', 'value'),
)

# The dataframe used here is the one I defined in feature2 
def map_choropleth(metric, year, df = df_feature2): # default year is 2022 
    
    # subset depending on which variable, metric and year the user wants to look at 
    if (year == 2020): 
        subset = df[df['date'] == date_year1].copy()
        
    elif (year == 2021): 
        subset = df[df['date'] == date_year2].copy()

    else: 
        subset = df[df['date'] == date_year3].copy()
        
    
    # Choropleth 
    fig = px.choropleth(
        subset,                               
        locations = 'iso_code',                  
        color = metric,                  
        color_continuous_scale = 'sunsetdark',    # arbitrary
        hover_name = 'country',              
        hover_data = {'iso_code' : False},
    )

    fig.update_layout(
        title = 'Hello', 
        coloraxis_colorbar_title = None,
        geo_showframe = False,
        margin = {'l' : 0, 'r' : 0, 'b' : 0, 't' : 0}
    )
    
    return fig

# ----------------------------------------------
# Callback for launching the bar-plot. Output depending on the user choice of radioitem

@app.callback(
    Output('my_barplot', 'figure'),
    Input('my_bar_choice', 'value'),
)

# The dataframe used here is the one I defined in feature3 
def barplot(choice, df = weekly_df): 
    
    subset = df.copy()
    
    fig = px.bar(
        subset,
        x = 'week',
        y = choice,
        color = choice
        )   

    fig.update_layout(
        xaxis_title = 'Week', 
        yaxis_title = None,
        title = f'Weekly occurences of {choice} in the world',
        title_x = 0.5
        )
    
    return fig


# ----------------------------------------------
# Launching the app 
if __name__ == '__main__':
    app.run_server(debug=True, port=8063)